# <center> Poisson Fourier Neural Operator training for sparse2dense experiments</center>

In this notebook we train the PoissonNet for the experiment sparse2dense where we test its performance when varying the sampling rate and the gaussian smoothing parameter

## Load libraries

In [1]:
# Utilities from dave
import sys
import os
sys.path.append('../../fourier_neural_operator/')

In [2]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from utilities3 import *

import operator
from functools import reduce
from functools import partial

from timeit import default_timer

from Adam import Adam

torch.manual_seed(0)
np.random.seed(0)
import gc

In [3]:
from importlib import reload
import utils
reload(utils)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


<module 'utils' from '/scratch/Github_repos/DeepShapeRecon/FNO_sparse2dense_sampling/utils.py'>

## Data loader

In [4]:
width = 30
modes = 10
training_steps = 10000
learning_rate = 0.0025
scheduler_step = 100
scheduler_gamma = 0.5

In [5]:
# root_path = '/scratch/Data/Fourier_Neural_Data/Sparse2Dense/'
# batch_size = 20
# n_samples = 5000
# sigma = 2

root_path = '/scratch/Data/Fourier_Neural_Data/Sparse2Dense/'
batch_size = 20
n_samples = 10000
sigma = 1.5

# root_path = '/scratch/Data/Fourier_Neural_Data/Sparse2Dense/'
# batch_size = 20
# n_samples = 25000
# sigma = 1.0

# root_path = '/scratch/Data/Fourier_Neural_Data/Sparse2Dense/'
# batch_size = 20
# n_samples = 50000
# sigma = 0.7

# root_path = '/scratch/Data/Fourier_Neural_Data/Sparse2Dense/'
# batch_size = 20
# n_samples = 100000
# sigma = 0.5

In [6]:
loader_train = utils.ShapeNet_FNO_sparse2dense_DataLoader(root_path, n_samples, sigma, 'train', batch_size)

/scratch/Github_repos/DeepShapeRecon/FNO_sparse2dense_sampling/utils.py:1787: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  self.files_name = np.array(np.array([x for x in


## PoissonNet model

In [7]:
device = torch.device('cuda')
model = utils.FNO3d(modes, modes, modes, width).cuda()

In [8]:
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=scheduler_step, gamma=scheduler_gamma)

In [9]:
checkpoint_path = '/scratch/Data/Fourier_Neural_Data/Sparse2Dense/checkpoints/n_samples_'+str(n_samples)+'_sigma_'+str(sigma)

In [10]:
best_loss = 10.20
if 1:
    model.load_state_dict(torch.load(checkpoint_path))

In [ ]:
myloss = LpLoss(size_average=False)
train_mse = 0
train_l2 = 0
S = T_in = T = 64 
for i in range(10000):
    model.train()
    # Loading data
    voxels_batch, divergence_tensor_batch = loader_train.generate_data()
    
    voxels_batch = torch.Tensor(voxels_batch).cuda()
    divergence_tensor_batch = torch.Tensor(divergence_tensor_batch).cuda()
    
    div_normalizer = UnitGaussianNormalizer(divergence_tensor_batch)
    divergence_tensor_batch = div_normalizer.encode(divergence_tensor_batch)
    
    vox_normalizer = UnitGaussianNormalizer(voxels_batch)
    voxels_batch = vox_normalizer.encode(voxels_batch)
    
    divergence_tensor_batch_resh = divergence_tensor_batch.reshape(batch_size,S,S,1,T_in).repeat([1,1,1,T,1])
    
    optimizer.zero_grad()
    pred = model(divergence_tensor_batch_resh).view(batch_size, S, S, T)

    mse = F.mse_loss(pred, voxels_batch, reduction='mean')
    
    voxels_batch = vox_normalizer.decode(voxels_batch)
    pred = div_normalizer.decode(pred)
    l2 = myloss(pred.view(batch_size, -1), voxels_batch.view(batch_size, -1))
    
    gc.collect()
    torch.cuda.empty_cache()
    
    print('Training step ', i, ' of ', 100000, ' with loss ', l2.cpu().detach().numpy())
    
    if l2 < best_loss:
        print("Saving checkpoints")
        torch.save(model.state_dict(), checkpoint_path)
        best_loss = l2
        
    l2.backward()

    optimizer.step()
    scheduler.step()
    gc.collect()
    torch.cuda.empty_cache()

Training step  0  of  100000  with loss  13.239762306213379
Training step  1  of  100000  with loss  10.986595153808594
Training step  2  of  100000  with loss  12.421419143676758
Training step  3  of  100000  with loss  12.48019027709961
Training step  4  of  100000  with loss  11.448732376098633
Training step  5  of  100000  with loss  13.71816635131836
Training step  6  of  100000  with loss  13.548208236694336
Training step  7  of  100000  with loss  12.164697647094727
Training step  8  of  100000  with loss  11.237825393676758
Training step  9  of  100000  with loss  13.851476669311523
Training step  10  of  100000  with loss  12.168966293334961
Training step  11  of  100000  with loss  12.655155181884766
Training step  12  of  100000  with loss  12.428014755249023
Training step  13  of  100000  with loss  12.21209716796875
Training step  14  of  100000  with loss  10.992338180541992
Training step  15  of  100000  with loss  11.740423202514648
Training step  16  of  100000  with l

Training step  135  of  100000  with loss  10.471293449401855
Training step  136  of  100000  with loss  11.782316207885742
Training step  137  of  100000  with loss  10.681689262390137
Training step  138  of  100000  with loss  9.637889862060547
Saving checkpoints
Training step  139  of  100000  with loss  10.269527435302734
Training step  140  of  100000  with loss  11.18637466430664
Training step  141  of  100000  with loss  10.074653625488281
Training step  142  of  100000  with loss  10.583532333374023
Training step  143  of  100000  with loss  9.431962966918945
Saving checkpoints
Training step  144  of  100000  with loss  9.453712463378906
Training step  145  of  100000  with loss  10.607980728149414
Training step  146  of  100000  with loss  9.78826904296875
Training step  147  of  100000  with loss  10.409717559814453
Training step  148  of  100000  with loss  10.234540939331055
Training step  149  of  100000  with loss  11.314552307128906
Training step  150  of  100000  with l

Training step  267  of  100000  with loss  10.566181182861328
Training step  268  of  100000  with loss  10.414207458496094
Training step  269  of  100000  with loss  10.80898666381836
Training step  270  of  100000  with loss  10.01443099975586
Training step  271  of  100000  with loss  10.365194320678711
Training step  272  of  100000  with loss  10.374505996704102
Training step  273  of  100000  with loss  10.451618194580078
Training step  274  of  100000  with loss  10.01776123046875
Training step  275  of  100000  with loss  10.037239074707031
Training step  276  of  100000  with loss  9.725468635559082
Training step  277  of  100000  with loss  9.568548202514648
Training step  278  of  100000  with loss  11.644128799438477
Training step  279  of  100000  with loss  9.700618743896484
Training step  280  of  100000  with loss  9.637807846069336
Training step  281  of  100000  with loss  12.62130355834961
Training step  282  of  100000  with loss  10.836769104003906
Training step  2

Training step  400  of  100000  with loss  9.47709846496582
Training step  401  of  100000  with loss  10.682880401611328
Training step  402  of  100000  with loss  10.098819732666016
Training step  403  of  100000  with loss  8.043462753295898
Saving checkpoints
Training step  404  of  100000  with loss  9.63914680480957
Training step  405  of  100000  with loss  10.236284255981445
Training step  406  of  100000  with loss  10.53149700164795
Training step  407  of  100000  with loss  10.401326179504395
Training step  408  of  100000  with loss  8.965582847595215
Training step  409  of  100000  with loss  9.32552719116211
Training step  410  of  100000  with loss  9.230177879333496
Training step  411  of  100000  with loss  9.355731964111328
Training step  412  of  100000  with loss  10.175984382629395
Training step  413  of  100000  with loss  10.435591697692871
Training step  414  of  100000  with loss  9.087166786193848
Training step  415  of  100000  with loss  9.205066680908203
Tr

Training step  534  of  100000  with loss  9.500150680541992
Training step  535  of  100000  with loss  11.083949089050293
Training step  536  of  100000  with loss  9.763446807861328
Training step  537  of  100000  with loss  10.088297843933105
Training step  538  of  100000  with loss  10.614084243774414
Training step  539  of  100000  with loss  10.134323120117188
Training step  540  of  100000  with loss  10.985052108764648
Training step  541  of  100000  with loss  11.181102752685547
Training step  542  of  100000  with loss  10.012157440185547
Training step  543  of  100000  with loss  10.486190795898438
Training step  544  of  100000  with loss  9.9282808303833
Training step  545  of  100000  with loss  9.486743927001953
Training step  546  of  100000  with loss  8.757387161254883
Training step  547  of  100000  with loss  10.502485275268555
Training step  548  of  100000  with loss  9.845500946044922
Training step  549  of  100000  with loss  10.495412826538086
Training step  5

...

Training step  5293  of  100000  with loss  9.469756126403809
Training step  5294  of  100000  with loss  10.037816047668457
Training step  5295  of  100000  with loss  11.372489929199219
Training step  5296  of  100000  with loss  11.019546508789062
Training step  5297  of  100000  with loss  9.047698974609375
Training step  5298  of  100000  with loss  10.075946807861328
Training step  5299  of  100000  with loss  10.809353828430176
Training step  5300  of  100000  with loss  9.214555740356445
Training step  5301  of  100000  with loss  12.078831672668457
Training step  5302  of  100000  with loss  10.1680269241333
Training step  5303  of  100000  with loss  8.455167770385742
Training step  5304  of  100000  with loss  10.604525566101074
Training step  5305  of  100000  with loss  10.720281600952148
Training step  5306  of  100000  with loss  9.571722984313965
Training step  5307  of  100000  with loss  9.496469497680664
Training step  5308  of  100000  with loss  8.890790939331055
T

Training step  5425  of  100000  with loss  9.826560020446777
Training step  5426  of  100000  with loss  9.788091659545898
Training step  5427  of  100000  with loss  9.643242835998535
Training step  5428  of  100000  with loss  9.333488464355469
Training step  5429  of  100000  with loss  8.572295188903809
Training step  5430  of  100000  with loss  9.36591911315918
Training step  5431  of  100000  with loss  9.362724304199219
Training step  5432  of  100000  with loss  8.811410903930664
Training step  5433  of  100000  with loss  9.968683242797852
Training step  5434  of  100000  with loss  10.630759239196777
Training step  5435  of  100000  with loss  9.282407760620117
Training step  5436  of  100000  with loss  10.40601634979248
Training step  5437  of  100000  with loss  10.739561080932617
Training step  5438  of  100000  with loss  10.857486724853516
Training step  5439  of  100000  with loss  10.628717422485352
Training step  5440  of  100000  with loss  10.92681884765625
Train

Training step  5557  of  100000  with loss  9.559284210205078
Training step  5558  of  100000  with loss  8.446537971496582
Training step  5559  of  100000  with loss  9.457221984863281
Training step  5560  of  100000  with loss  10.043050765991211
Training step  5561  of  100000  with loss  10.902681350708008
Training step  5562  of  100000  with loss  11.436840057373047
Training step  5563  of  100000  with loss  9.982864379882812
Training step  5564  of  100000  with loss  9.636985778808594
Training step  5565  of  100000  with loss  9.84273910522461
Training step  5566  of  100000  with loss  9.967805862426758
Training step  5567  of  100000  with loss  9.737488746643066
Training step  5568  of  100000  with loss  8.90349006652832
Training step  5569  of  100000  with loss  10.233125686645508
Training step  5570  of  100000  with loss  10.73939323425293
Training step  5571  of  100000  with loss  10.039070129394531
Training step  5572  of  100000  with loss  8.535747528076172
Train

Training step  5689  of  100000  with loss  10.020195960998535
Training step  5690  of  100000  with loss  10.415895462036133
Training step  5691  of  100000  with loss  9.710322380065918
Training step  5692  of  100000  with loss  10.445195198059082
Training step  5693  of  100000  with loss  11.080669403076172
Training step  5694  of  100000  with loss  9.849540710449219
Training step  5695  of  100000  with loss  9.310627937316895
Training step  5696  of  100000  with loss  10.86663818359375
Training step  5697  of  100000  with loss  10.33059024810791
Training step  5698  of  100000  with loss  9.949666976928711
Training step  5699  of  100000  with loss  10.346395492553711
Training step  5700  of  100000  with loss  8.49976921081543
Training step  5701  of  100000  with loss  10.295238494873047
Training step  5702  of  100000  with loss  9.084062576293945
Training step  5703  of  100000  with loss  11.048797607421875
Training step  5704  of  100000  with loss  10.931863784790039
T

Training step  5821  of  100000  with loss  10.228574752807617
Training step  5822  of  100000  with loss  10.853724479675293
Training step  5823  of  100000  with loss  9.735176086425781
Training step  5824  of  100000  with loss  10.913928985595703
Training step  5825  of  100000  with loss  11.407281875610352
Training step  5826  of  100000  with loss  10.692718505859375
Training step  5827  of  100000  with loss  9.115845680236816
Training step  5828  of  100000  with loss  10.024150848388672
Training step  5829  of  100000  with loss  9.964813232421875
Training step  5830  of  100000  with loss  9.905853271484375
Training step  5831  of  100000  with loss  9.999879837036133
Training step  5832  of  100000  with loss  10.846918106079102
Training step  5833  of  100000  with loss  9.30868148803711
Training step  5834  of  100000  with loss  10.138010025024414
Training step  5835  of  100000  with loss  10.4908447265625
Training step  5836  of  100000  with loss  10.60083293914795
Tr